# Phase 1: Data Ingestion & Exploration

**Objective**: Download and explore French open data sources for Demo A

**Date**: 2026-01-12

## Data Sources Pipeline

1. **DVF** (Demandes de Valeurs Foncières) - Primary transaction data
2. **API Adresse** (Base Adresse Nationale) - Geocoding validation
3. **DVF géolocalisées** - Pre-geocoded DVF alternative
4. **Cadastre** - Property reference data
5. **BDNB** - Building characteristics
6. **RNIC** - Co-ownership registry

**Sample scope**: Paris (75), 2023 full year

## Documentation Reference

**Notice descriptive** (column definitions): [PDF](https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-153319/notice-descriptive-du-fichier-dvf-20221017.pdf)

Consult this when you need to understand:
- What a specific column means
- Valid values/codes for categorical columns
- Business rules (why nulls exist, etc.)!

In [1]:
import json
from pathlib import Path

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import requests

# Setup directories
BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data" / "raw"
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")

Base directory: /home/iyid/workspaces/Certus-Labs/certus-showcase-a
Data directory: /home/iyid/workspaces/Certus-Labs/certus-showcase-a/data/raw


## 1. DVF (Demandes de Valeurs Foncières) - Primary Source

**Publisher**: Ministères économiques et financiers via data.gouv.fr  
**What**: French real estate transaction records  
**License**: Licence Ouverte  
**Coverage**: 2014-present, mainland France  

**Goal**: Download Paris (75) transactions for 2023

In [2]:
# Search for DVF dataset (don't hardcode ID)
SEARCH_URL = "https://www.data.gouv.fr/api/1/datasets/"
response = requests.get(SEARCH_URL, params={"q": "demandes valeurs foncieres", "page_size": 10})
response.raise_for_status()

results = response.json()
print(f"Found {results.get('total', 0)} datasets matching 'demandes valeurs foncieres'\n")

for dataset in results.get("data", []):
    print(f"Title: {dataset['title']}")
    print(f"ID: {dataset['id']}")
    print(f"Page: {dataset['page']}")
    print()

Found 4 datasets matching 'demandes valeurs foncieres'

Title: Demandes de valeurs foncières
ID: 5c4ae55a634f4117716d5656
Page: https://www.data.gouv.fr/datasets/demandes-de-valeurs-foncieres

Title: Demandes de valeurs foncières géolocalisées
ID: 5cc1b94a634f4165e96436c1
Page: https://www.data.gouv.fr/datasets/demandes-de-valeurs-foncieres-geolocalisees

Title: Demandes de valeurs foncières - Côtes d'Armor
ID: 67c035038c131cf1eabeb620
Page: https://www.data.gouv.fr/datasets/demandes-de-valeurs-foncieres-cotes-darmor

Title: Demandes de valeurs foncières géolocalisées sur le territoire de Saintes Grandes Rives, l'Agglo
ID: 68e7a944ec894b4ba3e5a3c5
Page: https://www.data.gouv.fr/datasets/demandes-de-valeurs-foncieres-geolocalisees-sur-le-territoire-de-saintes-grandes-rives-lagglo



In [3]:
# # Get resources from the main DVF dataset
# DVF_ID = "5c4ae55a634f4117716d5656"
# response = requests.get(f"https://www.data.gouv.fr/api/1/datasets/{DVF_ID}/")
# response.raise_for_status()

# dataset = response.json()
# resources = dataset.get('resources', [])

# print(f"Dataset: {dataset['title']}")
# print(f"Resources: {len(resources)}\n")

# for i, r in enumerate(resources):
#     print(f"[{i}] {r.get('title', 'N/A')}")
#     print(f"    Format: {r.get('format', 'N/A')}")
#     print(f"    URL: {r.get('url', 'N/A')}")
#     print()


# Get resources from the main DVF dataset
DVF_ID = "5c4ae55a634f4117716d5656"
response = requests.get(f"https://www.data.gouv.fr/api/1/datasets/{DVF_ID}/")
response.raise_for_status()

dataset = response.json()
resources = dataset.get("resources", [])

print(f"Dataset: {dataset['title']}")
print(f"Resources: {len(resources)}\n")

# Separate data files from documentation
data_files = []
doc_files = []

for r in resources:
    title = r.get("title", "")
    fmt = r.get("format", "")
    if fmt in ["pdf", "doc", "docx"] or "notice" in title.lower() or "faq" in title.lower():
        doc_files.append(r)
    else:
        data_files.append(r)

print("=== DATA FILES ===")
for i, r in enumerate(data_files):
    print(f"[{i}] {r.get('title')}")
    print(f"    Format: {r.get('format')}")
    print()

print("\n=== DOCUMENTATION ===")
for i, r in enumerate(doc_files):
    print(f"[{i}] {r.get('title')}")
    print(f"    Format: {r.get('format')}")
    print(f"    URL: {r.get('url')}")
    print()

Dataset: Demandes de valeurs foncières
Resources: 10

=== DATA FILES ===
[0] Valeurs foncières 2025 - Premier semestre
    Format: txt.zip

[1] Valeurs foncières 2024
    Format: txt.zip

[2] Valeurs foncières 2023
    Format: txt.zip

[3] Valeurs foncières 2022
    Format: txt.zip

[4] Valeurs foncières 2021
    Format: txt.zip

[5] Valeurs foncières 2020 - Second semestre
    Format: txt.zip


=== DOCUMENTATION ===
[0] Foire aux questions
    Format: pdf
    URL: https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-153257/faq-20221017.pdf

[1] Conditions générales d'utilisation
    Format: pdf
    URL: https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20201026-172435/conditions-generales-dutilisation-20201016.pdf

[2] Notice descriptive des fichiers de valeurs foncières
    Format: pdf
    URL: https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-153319/notice-descriptive-du-fichier-dvf-20221017.pdf

[3] Information de

In [4]:
# List ALL resources with original indexes
print("=== ALL RESOURCES ===")
for i, r in enumerate(resources):
    print(f"[{i}] {r.get('title', 'N/A')}")
    print(f"    Format: {r.get('format', 'N/A')}")
    if r.get("format") in ["pdf", "doc"]:
        print(f"    URL: {r.get('url')}")
    print()

=== ALL RESOURCES ===
[0] Valeurs foncières 2025 - Premier semestre
    Format: txt.zip

[1] Valeurs foncières 2024
    Format: txt.zip

[2] Valeurs foncières 2023
    Format: txt.zip

[3] Valeurs foncières 2022
    Format: txt.zip

[4] Valeurs foncières 2021
    Format: txt.zip

[5] Valeurs foncières 2020 - Second semestre
    Format: txt.zip

[6] Foire aux questions
    Format: pdf
    URL: https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-153257/faq-20221017.pdf

[7] Conditions générales d'utilisation
    Format: pdf
    URL: https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20201026-172435/conditions-generales-dutilisation-20201016.pdf

[8] Notice descriptive des fichiers de valeurs foncières
    Format: pdf
    URL: https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-153319/notice-descriptive-du-fichier-dvf-20221017.pdf

[9] Information des personnes concernées par le traitement informatique
    Format: pdf
    

In [5]:
# Download DVF 2023
import zipfile

dvf_2023_url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20251018-234851/valeursfoncieres-2023.txt.zip"

print("Downloading DVF 2023...")
response = requests.get(dvf_2023_url, stream=True)
response.raise_for_status()

# Save zip file
dvf_dir = DATA_DIR / "dvf"
dvf_dir.mkdir(exist_ok=True)

zip_path = dvf_dir / "valeursfoncieres-2023.txt.zip"
with open(zip_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"✓ Downloaded: {zip_path}")
print(f"  Size: {zip_path.stat().st_size / 1024 / 1024:.2f} MB")

✓ Downloaded: /home/iyid/workspaces/Certus-Labs/certus-showcase-a/data/raw/dvf/valeursfoncieres-2023.txt.zip
  Size: 68.28 MB


In [6]:
# Extract and examine structure
with zipfile.ZipFile(zip_path, "r") as z:
    txt_file = z.namelist()[0]
    z.extractall(dvf_dir)

txt_path = dvf_dir / txt_file
print(f"✓ Extracted: {txt_path}")
print(f"  Size: {txt_path.stat().st_size / 1024 / 1024:.2f} MB")

# Read first 3 lines to see columns and sample data
print("\n=== Structure ===")
with open(txt_path, encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(f"Line {i}: {line.strip()[:300]}")

✓ Extracted: /home/iyid/workspaces/Certus-Labs/certus-showcase-a/data/raw/dvf/ValeursFoncieres-2023.txt
  Size: 485.49 MB

=== Structure ===
Line 0: Identifiant de document|Reference document|1 Articles CGI|2 Articles CGI|3 Articles CGI|4 Articles CGI|5 Articles CGI|No disposition|Date mutation|Nature mutation|Valeur fonciere|No voie|B/T/Q|Type de voie|Code voie|Voie|Code postal|Commune|Code departement|Code commune|Prefixe de section|Section|No
Line 1: |||||||000001|05/01/2023|Vente|1070000,00|184||ALL|0124|DES HETRES|1630|ST-GENIS-POUILLY|01|354||BD|334||29||||||||||1|3|Dépendance||0|0|||
Line 2: |||||||000001|05/01/2023|Vente|1070000,00|159||ALL|0124|DES HETRES|1630|ST-GENIS-POUILLY|01|354||BD|336||22||||||||||1|2|Appartement||233|8|||


In [7]:
# Load into DataFrame - NO type inference, keep as strings
# This preserves raw data for quality checks and controlled transformations
df = pd.read_csv(txt_path, sep="|", encoding="utf-8", dtype=str, keep_default_na=False)

print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print("All columns loaded as strings (dtype=str)")
print("\nColumns:")
for i, col in enumerate(df.columns):
    print(f"  [{i}] {col}")

Shape: 3,812,327 rows × 43 columns
All columns loaded as strings (dtype=str)

Columns:
  [0] Identifiant de document
  [1] Reference document
  [2] 1 Articles CGI
  [3] 2 Articles CGI
  [4] 3 Articles CGI
  [5] 4 Articles CGI
  [6] 5 Articles CGI
  [7] No disposition
  [8] Date mutation
  [9] Nature mutation
  [10] Valeur fonciere
  [11] No voie
  [12] B/T/Q
  [13] Type de voie
  [14] Code voie
  [15] Voie
  [16] Code postal
  [17] Commune
  [18] Code departement
  [19] Code commune
  [20] Prefixe de section
  [21] Section
  [22] No plan
  [23] No Volume
  [24] 1er lot
  [25] Surface Carrez du 1er lot
  [26] 2eme lot
  [27] Surface Carrez du 2eme lot
  [28] 3eme lot
  [29] Surface Carrez du 3eme lot
  [30] 4eme lot
  [31] Surface Carrez du 4eme lot
  [32] 5eme lot
  [33] Surface Carrez du 5eme lot
  [34] Nombre de lots
  [35] Code type local
  [36] Type local
  [37] Identifiant local
  [38] Surface reelle bati
  [39] Nombre pieces principales
  [40] Nature culture
  [41] Nature culture

In [8]:
# Check what's actually in the "numeric-looking" columns before deciding types
print("=== Investigating Ambiguous Columns ===\n")

ambiguous_cols = ["No voie", "Code postal", "Code type local", "Code commune"]

for col in ambiguous_cols:
    print(f"\n{'=' * 60}")
    print(f"{col}")
    print("=" * 60)

    # Get non-null values
    values = df[col].dropna()

    # Show unique count
    n_unique = values.nunique()
    print(f"Unique values: {n_unique:,}")

    # Show sample of distinct values (first 20)
    distinct = values.unique()[:20]
    print(f"Sample: {distinct}")

    # Check if ALL are numeric
    try:
        numeric_test = pd.to_numeric(values, errors="coerce")
        n_non_numeric = numeric_test.isna().sum()
        pct_numeric = (1 - n_non_numeric / len(values)) * 100
        print(f"Numeric: {pct_numeric:.2f}% ({n_non_numeric:,} non-numeric values)")

        if n_non_numeric > 0:
            # Show non-numeric examples
            non_numeric = values[numeric_test.isna()].unique()[:10]
            print(f"Non-numeric examples: {non_numeric}")
    except Exception as e:
        print(f"Cannot test numericity: {e}")

print("\n\n" + "=" * 60)
print("CONCLUSION")
print("=" * 60)
print("Based on the above, decide:")
print("- String (identifiers/codes): Keep as-is, even if all numeric")
print("- Int64 (quantities): Convert to nullable integer for math operations")

=== Investigating Ambiguous Columns ===


No voie
Unique values: 7,267
Sample: ['184' '159' '2914' '427' '' '41' '352' '51' '6021' '212' '50' '270' '6'
 '33' '44' '7' '86' '5' '4' '228']
Numeric: 61.85% (1,454,312 non-numeric values)
Non-numeric examples: ['']

Code postal
Unique values: 5,867
Sample: ['1630' '1450' '1800' '1210' '1480' '1500' '1340' '1130' '1250' '1000'
 '1140' '1160' '1170' '1230' '1290' '1370' '1700' '1190' '1090' '1851']
Numeric: 99.33% (25,690 non-numeric values)
Non-numeric examples: ['']

Code type local
Unique values: 5
Sample: ['3' '2' '1' '' '4']
Numeric: 58.28% (1,590,408 non-numeric values)
Non-numeric examples: ['']

Code commune
Unique values: 907
Sample: ['354' '404' '361' '281' '446' '4' '24' '152' '408' '53' '348' '314' '422'
 '173' '420' '95' '416' '46' '378' '184']
Numeric: 100.00% (0 non-numeric values)


CONCLUSION
Based on the above, decide:
- String (identifiers/codes): Keep as-is, even if all numeric
- Int64 (quantities): Convert to nullable int

In [9]:
# Apply type transformations from expected schema
print("=== Applying Type Transformations ===\n")

# Replace empty strings with NaN for proper null handling
df = df.replace("", pd.NA)

# Define transformations based on Notice descriptive
# IMPORTANT: Distinguish between:
#   - Quantitative Int64: Can do math (count, sum, average)
#   - Categorical string: Identifiers/codes (even if all numeric)
#   - Date: Parse dates
#   - Float: Actual decimals (money, surfaces)

transformations = {
    # === Identifiers (keep as string) ===
    "No disposition": "int",  # Transaction ID - could argue for string, but appears to be sequential
    # === Dates ===
    "Date mutation": "date",  # Parse DD/MM/YYYY
    # === Monetary values (float) ===
    "Valeur fonciere": "float",  # Price in euros (comma decimal)
    # === Addresses/Codes (keep as string - may have non-numeric formats) ===
    "No voie": "str",  # Street number: "42", "44bis", etc.
    "Code postal": "str",  # Postal code: "75001", "CEDEX 92000", etc.
    "Code voie": "str",  # Internal code (already string)
    "Code commune": "str",  # Commune code (identifier, not a quantity)
    "Code departement": "str",  # Already string
    # === Cadastral identifiers (keep as string) ===
    "No plan": "str",  # Parcel number (identifier)
    "Section": "str",  # Already string
    # === Property type codes (categorical string) ===
    "Code type local": "str",  # 1=Maison, 2=Appt (categorical, not quantitative)
    # === Quantitative integers (Int64 - can do math) ===
    "Nombre de lots": "int",  # Count - can sum across transactions
    "Nombre pieces principales": "int",  # Count - can average, etc.
    # === Surfaces (float - can have decimals) ===
    "Surface reelle bati": "float",  # 123.45 m²
    "Surface terrain": "float",  # 1234.56 m²
}

for col, dtype in transformations.items():
    try:
        if dtype == "str":
            # Keep as string (already loaded as string, just confirm)
            df[col] = df[col].astype("object")
            print(f"✓ {col}: string (kept as-is)")
        elif dtype == "int":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
            print(f"✓ {col}: Int64")
        elif dtype == "float":
            # Handle comma decimal separator
            if col == "Valeur fonciere":
                df[col] = df[col].str.replace(",", ".", regex=False)
            df[col] = pd.to_numeric(df[col], errors="coerce")
            print(f"✓ {col}: float64")
        elif dtype == "date":
            df[col] = pd.to_datetime(df[col], format="%d/%m/%Y", errors="coerce")
            print(f"✓ {col}: datetime64")
    except Exception as e:
        print(f"✗ {col}: {e}")

print("\n✓ Transformations complete")
print("✓ String columns remain as-is (object dtype)")

=== Applying Type Transformations ===

✓ No disposition: Int64
✓ Date mutation: datetime64
✓ Valeur fonciere: float64
✓ No voie: string (kept as-is)
✓ Code postal: string (kept as-is)
✓ Code voie: string (kept as-is)
✓ Code commune: string (kept as-is)
✓ Code departement: string (kept as-is)
✓ No plan: string (kept as-is)
✓ Section: string (kept as-is)
✓ Code type local: string (kept as-is)
✓ Nombre de lots: Int64
✓ Nombre pieces principales: Int64
✓ Surface reelle bati: float64
✓ Surface terrain: float64

✓ Transformations complete
✓ String columns remain as-is (object dtype)


In [10]:
# Export to parquet using pyarrow native API (handles Int64 properly)
PROCESSED_DIR = BASE_DIR / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

output_path = PROCESSED_DIR / "dvf_2023.parquet"

print(f"Exporting to: {output_path.name}...")

# Convert DataFrame to Arrow table (preserves Int64 and all pandas types)
table = pa.Table.from_pandas(df, preserve_index=False)

# Write to parquet
pq.write_table(table, output_path, compression='snappy')

print(f"\n✓ Exported: {output_path}")
print(f"  Size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")
print(f"  Rows: {df.shape[0]:,}")
print(f"  Columns: {df.shape[1]}")
print("\n✓ Int64 types preserved in parquet")

Exporting to: dvf_2023.parquet...

✓ Exported: /home/iyid/workspaces/Certus-Labs/certus-showcase-a/data/processed/dvf_2023.parquet
  Size: 74.64 MB
  Rows: 3,812,327
  Columns: 43

✓ Int64 types preserved in parquet


In [11]:
# Verify parquet can be read back correctly
df_test = pd.read_parquet(output_path)

print("=== Parquet Verification ===\n")
print(f"Shape: {df_test.shape}")
print(f"Memory: {df_test.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"\nDate range: {df_test['Date mutation'].min()} to {df_test['Date mutation'].max()}")
print("\nSample dtypes:")
print(f"  No disposition: {df_test['No disposition'].dtype}")
print(f"  Date mutation: {df_test['Date mutation'].dtype}")
print(f"  No voie: {df_test['No voie'].dtype}")
print(f"  Code postal: {df_test['Code postal'].dtype}")
print(f"  Nombre de lots: {df_test['Nombre de lots'].dtype}")
print(f"  Valeur fonciere: {df_test['Valeur fonciere'].dtype}")

print("\n✓ Parquet file verified")

=== Parquet Verification ===

Shape: (3812327, 43)
Memory: 4754.07 MB

Date range: 2023-01-01 00:00:00 to 2023-12-31 00:00:00

Sample dtypes:
  No disposition: Int64
  Date mutation: datetime64[ns]
  No voie: object
  Code postal: object
  Nombre de lots: Int64
  Valeur fonciere: float64

✓ Parquet file verified


In [12]:
# Basic profiling
print("=== Data Types & Null Counts ===")
info = pd.DataFrame(
    {
        "dtype": df.dtypes,
        "nulls": df.isnull().sum(),
        "null_%": (df.isnull().sum() / len(df) * 100).round(2),
    }
)
print(info)

# print("\n=== Sample Row ===")
# df.head(1).T

=== Data Types & Null Counts ===
                                     dtype    nulls  null_%
Identifiant de document             object  3812327  100.00
Reference document                  object  3812327  100.00
1 Articles CGI                      object  3812327  100.00
2 Articles CGI                      object  3812327  100.00
3 Articles CGI                      object  3812327  100.00
4 Articles CGI                      object  3812327  100.00
5 Articles CGI                      object  3812327  100.00
No disposition                       Int64        0    0.00
Date mutation               datetime64[ns]        0    0.00
Nature mutation                     object        0    0.00
Valeur fonciere                    float64    34187    0.90
No voie                             object  1454312   38.15
B/T/Q                               object  3644264   95.59
Type de voie                        object  1543184   40.48
Code voie                           object    25443    0.67
Voie   

In [13]:
# Categorize columns by sparsity with descriptions
print("=== COLUMN CATEGORIZATION BY NULL RATE ===\n")

# Column descriptions from Notice descriptive
col_desc = {
    "No disposition": "Transaction ID within document",
    "Date mutation": "Transaction date (DD/MM/YYYY)",
    "Nature mutation": "Transaction type (Vente, Échange, Expropriation, Adjudication)",
    "Valeur fonciere": "Price in euros (comma decimal)",
    "No voie": "Street number",
    "B/T/Q": "Building/Tower/Block indicator",
    "Type de voie": "Street type (RUE, AVE, ALL, etc.)",
    "Code voie": "Internal street code",
    "Voie": "Street name",
    "Code postal": "5-digit postal code",
    "Commune": "Commune name",
    "Code departement": "2-digit département code",
    "Code commune": "3-digit commune code within département",
    "Prefixe de section": "Cadastral section prefix",
    "Section": "2-letter cadastral section",
    "No plan": "Parcel number within section",
    "No Volume": "Volume number (condos)",
    "1er lot": "First lot number",
    "Surface Carrez du 1er lot": "Carrez surface of first lot (m²)",
    "2eme lot": "Second lot number",
    "Surface Carrez du 2eme lot": "Carrez surface of second lot (m²)",
    "3eme lot": "Third lot number",
    "Surface Carrez du 3eme lot": "Carrez surface of third lot (m²)",
    "4eme lot": "Fourth lot number",
    "Surface Carrez du 4eme lot": "Carrez surface of fourth lot (m²)",
    "5eme lot": "Fifth lot number",
    "Surface Carrez du 5eme lot": "Carrez surface of fifth lot (m²)",
    "Nombre de lots": "Number of lots in transaction",
    "Code type local": "Property type code (1=Maison, 2=Appartement, 3=Dépendance, 4=Local)",
    "Type local": "Property type label",
    "Identifiant local": "Local identifier (fiscal)",
    "Surface reelle bati": "Built surface (m²) - for buildings only",
    "Nombre pieces principales": "Number of main rooms - for buildings only",
    "Nature culture": "Land type code (T=terres, AB=terrain à bâtir, VI=vignes, etc.)",
    "Nature culture speciale": "Detailed land type",
    "Surface terrain": "Land surface (m²) - for land only",
    "Identifiant de document": "Document ID (not used)",
    "Reference document": "Document reference (not used)",
    "1 Articles CGI": "Tax code article 1 (not used)",
    "2 Articles CGI": "Tax code article 2 (not used)",
    "3 Articles CGI": "Tax code article 3 (not used)",
    "4 Articles CGI": "Tax code article 4 (not used)",
    "5 Articles CGI": "Tax code article 5 (not used)",
}

null_pct = df.isnull().sum() / len(df) * 100

# Build categorized table
data = []
for col in df.columns:
    pct = null_pct[col]
    if pct == 100:
        cat = "Dead"
    elif pct > 90:
        cat = "Sparse"
    elif pct >= 30:
        cat = "Partial"
    else:
        cat = "Core"

    data.append(
        {
            "Column": col,
            "Null %": f"{pct:.1f}%",
            "Category": cat,
            "Description": col_desc.get(col, "?"),
        }
    )

col_df = pd.DataFrame(data)

# Display by category
for cat in ["Core", "Partial", "Sparse", "Dead"]:
    cat_df = col_df[col_df["Category"] == cat]
    print(f"\n{'=' * 80}")
    print(f"{cat.upper()} ({len(cat_df)} columns)")
    print("=" * 80)
    print(cat_df[["Column", "Null %", "Description"]].to_string(index=False))

# Store for later use
categories = {
    "Core": col_df[col_df["Category"] == "Core"]["Column"].tolist(),
    "Partial": col_df[col_df["Category"] == "Partial"]["Column"].tolist(),
    "Sparse": col_df[col_df["Category"] == "Sparse"]["Column"].tolist(),
    "Dead": col_df[col_df["Category"] == "Dead"]["Column"].tolist(),
}

=== COLUMN CATEGORIZATION BY NULL RATE ===


CORE (13 columns)
          Column Null %                                                    Description
  No disposition   0.0%                                 Transaction ID within document
   Date mutation   0.0%                                  Transaction date (DD/MM/YYYY)
 Nature mutation   0.0% Transaction type (Vente, Échange, Expropriation, Adjudication)
 Valeur fonciere   0.9%                                 Price in euros (comma decimal)
       Code voie   0.7%                                           Internal street code
            Voie   0.7%                                                    Street name
     Code postal   0.7%                                            5-digit postal code
         Commune   0.0%                                                   Commune name
Code departement   0.0%                                       2-digit département code
    Code commune   0.0%                        3-digit commune code

In [ ]:
# Check if dataset has schema metadata
print("=== Schema Metadata Check ===\n")

# Check dataset-level schema
if dataset.get("schema"):
    print("Dataset has schema:")
    print(json.dumps(dataset["schema"], indent=2))
else:
    print("No dataset-level schema found")

# Check each resource for schema
print("\n=== Resource-level Schemas ===")
for i, r in enumerate(resources):
    if r.get("schema"):
        print(f"\n[{i}] {r.get('title')}")
        print(json.dumps(r["schema"], indent=2))

# Check if any have a format we recognize
schema_formats = [
    r for r in resources if r.get("format", "").lower() in ["json-schema", "xsd", "schema.json"]
]
if schema_formats:
    print("\n=== Formal Schema Files ===")
    for r in schema_formats:
        print(f"- {r.get('title')}: {r.get('url')}")
else:
    print("\n⚠️ No formal schema files found in resources")
    print("Schema must be inferred from Notice descriptive PDF")

=== Schema Metadata Check ===

No dataset-level schema found

=== Resource-level Schemas ===

[6] Foire aux questions
{
  "name": null,
  "url": null,
  "version": null
}

[7] Conditions générales d'utilisation
{
  "name": null,
  "url": null,
  "version": null
}

[8] Notice descriptive des fichiers de valeurs foncières
{
  "name": null,
  "url": null,
  "version": null
}

[9] Information des personnes concernées par le traitement informatique
{
  "name": null,
  "url": null,
  "version": null
}

⚠️ No formal schema files found in resources
Schema must be inferred from Notice descriptive PDF


: 

## 2. API Adresse (Base Adresse Nationale) - Geocoding Validation

**Publisher**: Base Adresse Nationale via api.gouv.fr  
**What**: French address geocoding and validation API  
**License**: Open  
**API Docs**: https://adresse.data.gouv.fr/api-doc/adresse  
**Rate limits**: 50 req/sec (search), 1 req/sec (CSV batch)

**Goal**: Test geocoding with sample DVF addresses

In [ ]:
# Download documentation for offline reference
notice_url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-153319/notice-descriptive-du-fichier-dvf-20221017.pdf"

doc_dir = DATA_DIR.parent / "docs"
doc_dir.mkdir(exist_ok=True)

notice_path = doc_dir / "notice-dvf.pdf"
if not notice_path.exists():
    response = requests.get(notice_url)
    response.raise_for_status()
    notice_path.write_bytes(response.content)
    print(f"✓ Downloaded: {notice_path}")
else:
    print(f"✓ Already exists: {notice_path}")

In [ ]:
# Test API Adresse with sample queries
api_adresse_base = "https://api-adresse.data.gouv.fr"

# Test single address search
test_address = "8 Boulevard du Palais 75001 Paris"
response = requests.get(f"{api_adresse_base}/search/", params={"q": test_address})
response.raise_for_status()

result = response.json()
print("=== API Adresse Test ===")
print(f"Query: {test_address}")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False)[:500])

# Check rate limit headers
print("\n=== Rate Limit Info ===")
for header, value in response.headers.items():
    if "rate" in header.lower() or "limit" in header.lower():
        print(f"{header}: {value}")

In [ ]:
# TODO: Research DVF géolocalisées dataset structure and download options
# Dataset page: https://www.data.gouv.fr/fr/datasets/...
#
# Steps:
# 1. Find dataset API endpoint or direct download URL
# 2. Check if Paris (75) data is available separately or in full France file
# 3. Download sample
# 4. Compare schema with raw DVF
# 5. Assess geocoding confidence/quality metrics

print("TODO: Explore DVF géolocalisées dataset")

## 4. Cadastre - Property Reference Data

**Publisher**: data.gouv.fr  
**What**: French cadastral (land registry) data  
**License**: Licence Ouverte  

**Goal**: Cross-reference property identifiers from DVF with cadastral parcels

**Status**: TODO - Enrichment source, explore after DVF quality issues identified

In [ ]:
# TODO: Research Cadastre data access
#
# Questions to answer:
# 1. What API or download format is available?
# 2. Can we filter by département (75)?
# 3. What identifiers link to DVF (parcel ID format)?
# 4. File size considerations for Paris
#
# Potential use case:
# - Validate DVF parcel IDs against official cadastre
# - Enrich with property characteristics (surface, type)
# - Detect inconsistencies in property identifiers

print("TODO: Explore Cadastre data source")

## 5. BDNB (Base de données nationale des bâtiments) - Building Characteristics

**Publisher**: Centre Scientifique et Technique du Batiment (CSTB) via data.gouv.fr  
**What**: National database of building characteristics  
**License**: Licence Ouverte  

**Goal**: Enrich DVF transactions with building-level data (energy, construction year, etc.)

**Status**: TODO - Advanced enrichment, explore if Phase 1 findings require building-level analysis

In [ ]:
# TODO: Research BDNB data structure and access methods
#
# Questions to answer:
# 1. What's the data format and size?
# 2. How to link BDNB to DVF (address? parcel ID? coordinates?)
# 3. What building characteristics are available and relevant?
# 4. API vs bulk download options
#
# Potential quality checks:
# - DVF construction year vs BDNB construction year
# - Energy performance class consistency
# - Building type validation (appartement vs maison)

print("TODO: Explore BDNB dataset")

## 6. RNIC (Registre National d'Immatriculation des Copropriétés) - Co-ownership Registry

**Publisher**: Agence Nationale de l'Habitat via data.gouv.fr  
**What**: National registry of co-ownership properties  
**License**: Licence Ouverte  

**Goal**: Identify co-ownership properties in DVF, cross-reference registration status

**Status**: TODO - Specialized enrichment, explore if co-ownership quality issues arise in Phase 1

In [ ]:
# TODO: Research RNIC data access and structure
#
# Questions to answer:
# 1. What format is RNIC data available in?
# 2. How to match RNIC to DVF (address? cadastral ID?)
# 3. What co-ownership attributes are provided?
# 4. Coverage and completeness (% of apartments registered)
#
# Potential quality checks:
# - DVF "appartement" transactions vs RNIC registration status
# - Co-ownership size/units consistency
# - Registration date vs transaction date coherence

print("TODO: Explore RNIC dataset")

## Next Steps

**Immediate (Phase 1 - Exploration)**:
1. ✅ Set up data ingestion notebook structure
2. 🔄 Run DVF API exploration (cell 4) to identify Paris 2023 file
3. 🔄 Download DVF Paris data
4. 🔄 Test API Adresse with sample queries
5. ⏸️ Load DVF data and begin profiling (schema, nulls, outliers)

**Phase 1 Continuation**:
- Explore DVF géolocalisées as alternative/comparison source
- Evaluate need for Cadastre, BDNB, RNIC based on initial DVF findings

**Decision Point**:
After initial DVF exploration, decide which enrichment sources are needed based on:
- Quality issues discovered
- Business rules requiring cross-validation
- Complexity vs. value trade-off

---

**Status Legend**:
- ✅ Complete
- 🔄 In progress
- ⏸️ Waiting / Next
- 📋 TODO